<a href="https://colab.research.google.com/github/mainguyenanhvu/Clinic_Managerment_Application/blob/master/Convert_ImageCLEF_SPLL_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!rm -r /content/spll
#!cd /content
!rm -r spll/data/preprocessed
!rm -r spll/data/merged
!mkdir spll/data/merged
!mkdir spll/data/preprocessed
!pwd

/content


#DOWNLOAD DATA

In [4]:
#@title Default title text
# Download data

!mkdir spll
!mkdir spll/data
!mkdir spll/data/unzip
!mkdir spll/data/converted
!mkdir spll/data/merged
!mkdir spll/data/preprocessed
!mkdir spll/data/database
%cd spll/data
!wget https://aicrowd-production.s3.eu-central-1.amazonaws.com/task_dataset_files/clef_task_26/09141252-f82d-4910-aac4-1c9e229cc324_ImageCLEF_SPLL_Data.zip
!unzip 09141252-f82d-4910-aac4-1c9e229cc324_ImageCLEF_SPLL_Data.zip -d ./unzip > unzip_log.txt
%cd ../..

/content/spll/data
--2020-04-22 23:39:45--  https://aicrowd-production.s3.eu-central-1.amazonaws.com/task_dataset_files/clef_task_26/09141252-f82d-4910-aac4-1c9e229cc324_ImageCLEF_SPLL_Data.zip
Resolving aicrowd-production.s3.eu-central-1.amazonaws.com (aicrowd-production.s3.eu-central-1.amazonaws.com)... 52.219.47.37
Connecting to aicrowd-production.s3.eu-central-1.amazonaws.com (aicrowd-production.s3.eu-central-1.amazonaws.com)|52.219.47.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1815394273 (1.7G) [binary/octet-stream]
Saving to: ‘09141252-f82d-4910-aac4-1c9e229cc324_ImageCLEF_SPLL_Data.zip’

09141252-f82d-4910- 100%[===================>]   1.69G  30.0MB/s    in 59s     

2020-04-22 23:40:45 (29.2 MB/s) - ‘09141252-f82d-4910-aac4-1c9e229cc324_ImageCLEF_SPLL_Data.zip’ saved [1815394273/1815394273]

mapname:  conversion of  failed
/content


#IMPORT LIBRARIES

In [0]:
# Import libraries

import json as js
from os import path, listdir, mkdir
import pandas as pd
import numpy as np
import csv
import re
import functools
import shutil

In [0]:
# Define constants

DATA_DIR = './spll/data/unzip'
CONVERTED_DIR = './spll/data/converted'
MERGED_DIR = './spll/data/merged'
PRE_PROCESSED_DIR = './spll/data/preprocessed'
DATABASE = './spll/data/database'

#CONVERT JSON TO CSV

##Calories file

In [0]:
# Convert files have the format which sames to calories files (calories, distance, steps, active minutes)

def convert_calories_format_data_file(infile, outfile):
  with open(infile) as source:
    calories = js.load(source)
    calories_df = pd.DataFrame(calories)
    calories_df.to_csv(outfile, header=True, index=False)

##Heart rate file

In [0]:
# Convert files have the format which sames to heart rate files

def convert_heart_rate_format_data_file(infile, outfile):
  with open(infile) as source:
    heart_rate = js.load(source)
    for value in heart_rate:
      value['bpm'] = value['value']['bpm']
      value['confidence'] = value['value']['confidence']
      value.pop('value', None)
    heart_rate_df = pd.DataFrame(heart_rate)
    heart_rate_df.to_csv(outfile, header=True, index=False)

##Heart rate zone file

In [0]:
# Convert files have the format which sames to heart rate zones files

def convert_heart_rate_zones_format_data_file(infile, outfile):
  with open(infile) as source:
    heart_rate_zones = js.load(source)
    for value in heart_rate_zones:
      value['IN_DEFAULT_ZONE_1'] = value['value']['valuesInZones']['IN_DEFAULT_ZONE_1']
      value['IN_DEFAULT_ZONE_2'] = value['value']['valuesInZones']['IN_DEFAULT_ZONE_2']
      value['IN_DEFAULT_ZONE_3'] = value['value']['valuesInZones']['IN_DEFAULT_ZONE_3']
      value['BELOW_DEFAULT_ZONE_1'] = value['value']['valuesInZones']['BELOW_DEFAULT_ZONE_1']
      value.pop('value', None)
    heart_rate_zones_df = pd.DataFrame(heart_rate_zones)
    heart_rate_zones_df.to_csv(outfile, header=True, index=False)

##Sleep file

In [0]:
# Convert files have the format which sames to sleep files

def convert_sleep_format_data_file(infile, sleep_outfile, level_outfile, data_outfile):
  with open(infile) as source:
    sleep = js.load(source)

    sleep_level_summaries = []
    sleep_level_datas = []
    for value in sleep:
      sleep_level_summaries.append(value['levels']['summary'])
      sleep_level_datas.append(value['levels']['data'])
      value.pop('levels', None)
      value.pop('logId', None)
    
    for i in range(len(sleep)):
      value = sleep_level_summaries[i]
      value['dateOfSleep'] = sleep[i]['dateOfSleep']

      try:
        value['deep_count'] = value['deep']['count']
        value['deep_minutes'] = value['deep']['minutes']
        value['deep_thirty_day_avg_minutes'] = value['deep']['thirtyDayAvgMinutes']

        value['wake_count'] = value['wake']['count']
        value['wake_minutes'] = value['wake']['minutes']
        value['wake_thirty_day_avg_minutes'] = value['wake']['thirtyDayAvgMinutes']

        value['light_count'] = value['light']['count']
        value['light_minutes'] = value['light']['minutes']
        value['light_thirty_day_avg_minutes'] = value['light']['thirtyDayAvgMinutes']

        value['rem_count'] = value['rem']['count']
        value['rem_minutes'] = value['rem']['minutes']
        value['rem_thirty_day_avg_minutes'] = value['rem']['thirtyDayAvgMinutes']

        value.pop('deep', None)
        value.pop('light', None)
        value.pop('rem', None)
        value.pop('wake', None)
      except KeyError:
        value['deep_count'] = value['deep_minutes'] = value['deep_thirty_day_avg_minutes'] = None

        value['wake_count'] = value['wake_minutes'] = value['wake_thirty_day_avg_minutes'] = None

        value['light_count'] = value['light_minutes'] = value['light_thirty_day_avg_minutes'] = None

        value['rem_count'] = value['rem_minutes'] = value['rem_thirty_day_avg_minutes'] = None

        value.pop('restless', None)
        value.pop('awake', None)
        value.pop('asleep', None)

      datas = sleep_level_datas[i]
      for data in datas:
        data['dateOfSleep'] = sleep[i]['dateOfSleep']

    sleep_level_datas = functools.reduce(lambda x, y: x+y, sleep_level_datas)

    sleep_level_summaries_df = pd.DataFrame(sleep_level_summaries)
    sleep_level_datas_df = pd.DataFrame(sleep_level_datas)
    sleep_df = pd.DataFrame(sleep)

    sleep_df.to_csv(sleep_outfile, header=True, index=False)
    sleep_level_summaries_df.to_csv(level_outfile, header=True, index=False)
    sleep_level_datas_df.to_csv(data_outfile, header=True, index=False)

##Custom heart rate file

In [0]:
# Convert files have the format which sames to custom heart rate files

def convert_custom_heart_rate_format_data_file(infile, outfile):
  with open(infile) as source:
    custom_heart_rate = js.load(source)
    for value in custom_heart_rate:
      try:
        value['enabled'] = value['value']['enabled']
        value['maxHeartRate'] = value['value']['maxHeartRate']
      except KeyError:
        try:
          value['minThreshold'] = value['value']['minThreshold']
          value['maxThreshold'] = value['value']['maxThreshold']
        except KeyError:
          value['error'] = value['value']['error']
          value['resting heart rate'] = value['value']['value']
      value.pop('value', None)
    custom_heart_rate_df = pd.DataFrame(custom_heart_rate)
    custom_heart_rate_df.to_csv(outfile, header=True, index=False)

##Exercise file

In [0]:
# Convert files have the format which sames to exercise files

def convert_exercise_format_data_file(infile, outfile):
  with open(infile) as source:
    exercise = js.load(source)
    
    for value in exercise:
      value['sedentary_active_minutes'] = [x for x in value['activityLevel'] if x['name']=='sedentary'][0]['minutes']
      value['lightly_active_minutes'] = [x for x in value['activityLevel'] if x['name']=='lightly'][0]['minutes']
      value['fairly_active_minutes'] = [x for x in value['activityLevel'] if x['name']=='fairly'][0]['minutes']
      value['very_active_minutes'] = [x for x in value['activityLevel'] if x['name']=='very'][0]['minutes']

      try:
        value['time_in_fat_burn_heart_rate_zone'] = [x for x in value['heartRateZones'] if x['name']=='Fat Burn'][0]['minutes']
        value['time_in_out_of_range_heart_rate_zone'] = [x for x in value['heartRateZones'] if x['name']=='Out of Range'][0]['minutes']
        value['time_in_peak_heart_rate_zone'] = [x for x in value['heartRateZones'] if x['name']=='Peak'][0]['minutes']
        value['time_in_cardio_heart_rate_zone'] = [x for x in value['heartRateZones'] if x['name']=='Cardio'][0]['minutes']
      except KeyError:
        value['time_in_fat_burn_heart_rate_zone'] = None
        value['time_in_out_of_range_heart_rate_zone'] = None
        value['time_in_peak_heart_rate_zone'] = None
        value['time_in_cardio_heart_rate_zone'] = None
      
      try:
        value['vo2Max'] = value['vo2Max']['vo2Max']
      except KeyError:
        value['vo2Max'] = None

      value.pop('logId', None)
      value.pop('source', None)
      value.pop('logType', None)
      value.pop('manualValuesSpecified', None)
      value.pop('tcxLink', None)
      value.pop('shouldFetchDetails', None)
      value.pop('hasGps', None)
      value.pop('activityTypeId', None)
      value.pop('activityLevel', None)
      value.pop('heartRateZones', None)

    exercise_df = pd.DataFrame(exercise)
    exercise_df.to_csv(outfile, header=True, index=False)

##Convert all file in fitbit folder

In [0]:
def convert_fitbit_folder():
  for d in listdir(DATA_DIR):
    d_path = path.join(DATA_DIR, d)
    if path.isfile(d_path):
      continue

    converted_d_path = path.join(CONVERTED_DIR, d)
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
      mkdir(path.join(converted_d_path,'fitbit'))
    
    d_path = path.join(d_path, 'fitbit')
    converted_d_path = path.join(converted_d_path, 'fitbit')
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue
      
      if re.search('^(calories|distance|steps|very_active_minutes|moderately_active_minutes|lightly_active_minutes|sedentary_minutes).*(json)$', f):
        convert_calories_format_data_file(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'))
      
      if re.search('^(heart_rate).*(json)$', f):
        convert_heart_rate_format_data_file(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'))

      if re.search('^(time_in_heart_rate_zones).*(json)$', f):
        convert_heart_rate_zones_format_data_file(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'))

      if re.search('^(custom|resting_heart_rate).*(json)$', f):
        convert_custom_heart_rate_format_data_file(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'))

      if re.search('^(sleep).*(json)$', f):
        sleep_outfile = path.join(converted_d_path, f.split('.')[0] + '.csv')
        level_outfile = path.join(converted_d_path, 'level_' + f.split('.')[0] + '.csv')
        data_outfile = path.join(converted_d_path, 'data_' + f.split('.')[0] + '.csv')
        convert_sleep_format_data_file(f_path, sleep_outfile, level_outfile, data_outfile)
      
      if re.search('^(exercise).*(json)$', f):
        convert_exercise_format_data_file(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'))

#Merge files in fitbit folder

In [0]:
# Merge files in fitbit folder

def merge_fitbit_folder():
  for d in listdir(CONVERTED_DIR):
    d_path = path.join(CONVERTED_DIR, d)
    if path.isfile(d_path):
      continue

    merged_d_path= path.join(MERGED_DIR, d)
    if not(path.exists(merged_d_path)):
      mkdir(merged_d_path)
    
    calories = []
    distances = []
    steps = []
    heart_rates = []
    very_active_minutes = []
    moderately_active_minutes = []
    lightly_active_minutes = []
    sedentary_minutes = []
    time_in_zones = []
    sleeps = []
    exercises = []
    level_sleeps = []
    data_sleeps = []

    # calories_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/calories-2019-10-08.csv')).columns)
    # distances_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/distance-2019-10-08.csv')).columns)
    # steps_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/steps-2019-10-08.csv')).columns)
    # heart_rates_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/heart_rate-2019-10-10.csv')).columns)
    # very_active_minutes_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/very_active_minutes-2019-10-08.csv')).columns)
    # moderately_active_minutes_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/moderately_active_minutes-2019-10-08.csv')).columns)
    # lightly_active_minutes_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/lightly_active_minutes-2019-10-08.csv')).columns)
    # sedentary_minutes_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/sedentary_minutes-2019-10-08.csv')).columns)
    # time_in_zones_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/time_in_heart_rate_zones-2019-10-10.csv')).columns)
    # sleeps_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/sleep-2019-10-08.csv')).columns)
    # exercises_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/exercise-0.csv')).columns)
    # level_sleeps_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/level_sleep-2019-10-08.csv')).columns)
    # data_sleeps_header = list(pd.read_csv(path.join(CONVERTED_DIR, 'p01/fitbit/data_sleep-2019-10-08.csv')).columns)

    d_path = path.join(d_path, 'fitbit')
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue

      df = pd.read_csv(f_path)
      
      if re.search('^(calories).*(csv)$', f):
        calories.append(df)
        
      if re.search('^(distance).*(csv)$', f):
        distances.append(df)

      if re.search('^(steps).*(csv)$', f):
        steps.append(df)

      if re.search('^(very_active_minutes).*(csv)$', f):
        very_active_minutes.append(df)

      if re.search('^(moderately_active_minutes).*(csv)$', f):
        moderately_active_minutes.append(df)

      if re.search('^(lightly_active_minutes).*(csv)$', f):
        lightly_active_minutes.append(df)

      if re.search('^(sedentary_minutes).*(csv)$', f):
        sedentary_minutes.append(df)
      
      if re.search('^(heart_rate).*(csv)$', f):
        heart_rates.append(df)

      if re.search('^(time_in_heart_rate_zones).*(csv)$', f):
        time_in_zones.append(df)

      if re.search('^(sleep).*(csv)$', f):
        sleeps.append(df)

      if re.search('^(level_sleep).*(csv)$', f):
        level_sleeps.append(df)

      if re.search('^(data_sleep).*(csv)$', f):
        data_sleeps.append(df)
      
      if re.search('^(exercise).*(csv)$', f):
        exercises.append(df)

    if len(sleeps) != 0:
      sleeps_df = pd.concat(sleeps)
      sleeps_df.to_csv(path.join(merged_d_path, 'sleep.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'sleep.csv'), 'w') as f:
    #     f.write(', '.join(sleeps_header))

    if len(level_sleeps) != 0:
      level_sleeps_df = pd.concat(level_sleeps)
      level_sleeps_df.to_csv(path.join(merged_d_path, 'level_sleep.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'level_sleep.csv'), 'w') as f:
    #     f.write(', '.join(level_sleeps_header))

    if len(sleeps) != 0:
      data_sleeps_df = pd.concat(data_sleeps)
      data_sleeps_df.to_csv(path.join(merged_d_path, 'data_sleep.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'data_sleep.csv'), 'w') as f:
    #     f.write(', '.join(data_sleeps_header))

    if len(exercises) != 0:
      exercises_df = pd.concat(exercises)
      exercises_df.to_csv(path.join(merged_d_path, 'exercise.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'exercise.csv'), 'w') as f:
    #     f.write(', '.join(exercises_header))

    if len(time_in_zones) != 0:
      time_in_zones_df = pd.concat(time_in_zones)
      time_in_zones_df.to_csv(path.join(merged_d_path, 'time_in_heart_rate_zones.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'time_in_heart_rate_zones.csv'), 'w') as f:
    #     f.write(', '.join(time_in_heart_rate_zones_header))

    if len(heart_rates) != 0:
      heart_rates_df = pd.concat(heart_rates)
      heart_rates_df.to_csv(path.join(merged_d_path, 'heart_rate.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'heart_rate.csv'), 'w') as f:
    #     f.write(', '.join(heart_rates_header))

    if len(sedentary_minutes) != 0:
      sedentary_minutes_df = pd.concat(sedentary_minutes)
      sedentary_minutes_df.to_csv(path.join(merged_d_path, 'sedentary_minutes.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'sedentary_minutes.csv'), 'w') as f:
    #     f.write(', '.join(sedentary_minutes_header))

    if len(lightly_active_minutes) != 0:
      lightly_active_minutes_df = pd.concat(lightly_active_minutes)
      lightly_active_minutes_df.to_csv(path.join(merged_d_path, 'lightly_active_minutes.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'lightly_active_minutes.csv'), 'w') as f:
    #     f.write(', '.join(lightly_active_minutes_header))

    if len(moderately_active_minutes) != 0:
      moderately_active_minutes_df = pd.concat(moderately_active_minutes)
      moderately_active_minutes_df.to_csv(path.join(merged_d_path, 'moderately_active_minutes.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'moderately_active_minutes.csv'), 'w') as f:
    #     f.write(', '.join(moderately_active_minutes_header))

    if len(very_active_minutes) != 0:
      very_active_minutes_df = pd.concat(very_active_minutes)
      very_active_minutes_df.to_csv(path.join(merged_d_path, 'very_active_minutes.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'very_active_minutes.csv'), 'w') as f:
    #     f.write(', '.join(very_active_minutes_header))

    if len(steps) != 0:
      steps_df = pd.concat(steps)
      steps_df.to_csv(path.join(merged_d_path, 'steps.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'steps.csv'), 'w') as f:
    #     f.write(', '.join(steps_header))

    if len(distances) != 0:
      distances_df = pd.concat(distances)
      distances_df.to_csv(path.join(merged_d_path, 'distance.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'distances.csv'), 'w') as f:
    #     f.write(', '.join(distances_header))

    if len(calories) != 0:
      calories_df = pd.concat(calories)
      calories_df.to_csv(path.join(merged_d_path, 'calories.csv'), header=True, index=False)
    # else:
    #   with open(path.join(merged_d_path, 'calories.csv'), 'w') as f:
    #     f.write(', '.join(calories_header))

In [0]:
# Merge multi filetypes into one

def merge_multi_file_type_of_fitbit_folder(in_dir, out_dir):
  for d in listdir(in_dir):
    d_path = path.join(in_dir, d)
    if path.isfile(d_path):
      continue
    
    calories_df = None
    distance_df = None
    steps_df = None
    heart_rate_df = None
    lightly_active_minutes_df = None
    very_active_minutes = None
    sedentary_minutes = None
    moderately_active_minutes = None

    
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue

      if re.search('^(calories).*(csv)$', f):
        calories_df = pd.read_csv(f_path)
        calories_df.rename(columns={'value': 'calories'}, inplace=True)
      
      if re.search('^(heart_rate).*(csv)$', f):
        heart_rate_df = pd.read_csv(f_path)
        heart_rate_df.rename(columns={'bpm': 'heart_rate', 'confidence': 'heart_rate_confidence'}, inplace=True)

      if re.search('^(distance).*(csv)$', f):
        distance_df = pd.read_csv(f_path)
        distance_df.rename(columns={'value': 'distance'}, inplace=True)
      
      if re.search('^(steps).*(csv)$', f):
        steps_df = pd.read_csv(f_path)
        steps_df.rename(columns={'value': 'steps'}, inplace=True)

      if re.search('^(very_active_minutes).*(csv)$', f):
        very_active_minutes = pd.read_csv(f_path)
        very_active_minutes.rename(columns={'value': 'very_active'}, inplace=True)

      if re.search('^(moderately_active_minutes).*(csv)$', f):
        moderately_active_minutes_df = pd.read_csv(f_path)
        moderately_active_minutes_df.rename(columns={'value': 'moderately_active'}, inplace=True)

      if re.search('^(lightly_active_minutes).*(csv)$', f):
        lightly_active_minutes_df = pd.read_csv(f_path)
        lightly_active_minutes_df.rename(columns={'value': 'lightly_active'}, inplace=True)

      if re.search('^(sedentary_minutes).*(csv)$', f):
        sedentary_minutes_df = pd.read_csv(f_path)
        sedentary_minutes_df.rename(columns={'value': 'sedentary'}, inplace=True)

    calories_df.drop_duplicates(subset='dateTime', keep='first', inplace=True)
    distance_df.drop_duplicates(subset='dateTime', keep='first', inplace=True)
    steps_df.drop_duplicates(subset='dateTime', keep='first', inplace=True)
    heart_rate_df.drop_duplicates(subset='dateTime', keep='first', inplace=True)

    out_path = path.join(out_dir, d)
    if not(path.exists(out_path)):
      mkdir(out_path)

    total_cal_dis_ste_hea_df = pd.concat([calories_df.set_index('dateTime'), distance_df.set_index('dateTime'), steps_df.set_index('dateTime'), heart_rate_df.set_index('dateTime')], axis=1)
    total_active_minutes_df = pd.concat([very_active_minutes.set_index('dateTime'), moderately_active_minutes_df.set_index('dateTime'), lightly_active_minutes_df.set_index('dateTime'), sedentary_minutes_df.set_index('dateTime')], axis=1)

    total_cal_dis_ste_hea_df.index.set_names('dateTime', inplace=True)
    total_active_minutes_df.index.set_names('dateTime', inplace=True)
    total_cal_dis_ste_hea_df.reset_index(inplace=True)
    total_active_minutes_df.reset_index(inplace=True)

    total_cal_dis_ste_hea_df.to_csv(path.join(out_path, 'total_cal_dis_ste_hea.csv'), header=True, index=False)
    total_active_minutes_df.to_csv(path.join(out_path, 'total_active_minutes.csv'), header=True, index=False)

#CONVER FILE IN PMSYS FOLDER

In [0]:
def convert_injury(infile,outfile,filename):
  print(outfile)
  filetxt = str(infile).replace(filename,'info_'+filename.split('.')[0]+'.txt')
  fi = open(filetxt,'r')
  columns = fi.readline().replace(' ','').split(';')
  fi.close()

  fo = open(outfile,'w')
  fo.write('{},{}\n'.format(columns[0],columns[1]))
  reader = csv.reader(open(infile,'r'))
  for row in reader:
    temp = str(row).replace(' ','').replace('[','').replace(']','').replace("'","").replace('"','').replace(',','|')
    #print(temp)
    arow = temp.split(';')
    fo.write('{},{}\n'.format(arow[0],arow[1]))
  fo.close()

In [0]:
def convert_srpe(infile,outfile,filename):
  print(outfile)
  filetxt = str(infile).replace(filename,'info_'+filename.split('.')[0]+'.txt')
  fi = open(filetxt,'r')
  columns = fi.readline().replace(' ','').split(';')
  fi.close()

  fo = open(outfile,'w')
  fo.write('{},{},{},{}\n'.format(columns[0],columns[1],columns[2],columns[3]))
  reader = csv.reader(open(infile,'r'))
  for row in reader:
    temp = str(row).replace(' ','').replace('[','').replace(']','').replace("'","").replace('"','').replace(',','|')
    #print(temp)
    arow = temp.split(';')
    fo.write('{},{},{},{}\n'.format(arow[0],arow[1],arow[3],arow[3]))
  fo.close()

In [0]:
def convert_wellness(infile,outfile,filename):
  print(outfile)
  filetxt = str(infile).replace(filename,'info_'+filename.split('.')[0]+'.txt')
  fi = open(filetxt,'r')
  columns = fi.readline().replace(' ','').split(';')
  fi.close()

  fo = open(outfile,'w')
  fo.write('{},{},{},{},{},{},{},{},{}\n'.format(columns[0],columns[1],columns[2],columns[3],columns[4],columns[5],columns[6],columns[7],columns[8]))
  reader = csv.reader(open(infile,'r'))
  for row in reader:
    temp = str(row).replace(' ','').replace('[','').replace(']','').replace("'","").replace('"','').replace(',','|')
    #print(temp)
    arow = temp.split(';')
    fo.write('{},{},{},{},{},{},{},{},{}\n'.format(arow[0],arow[1],arow[3],arow[3],arow[4],arow[5],arow[6],arow[7],arow[8]))
  fo.close()

In [0]:
def convert_pmsys_folder():
  #convert pmsys/[file]
  for d in listdir(DATA_DIR):
    d_path = path.join(DATA_DIR, d)
    if path.isfile(d_path):
      continue

    converted_d_path = path.join(CONVERTED_DIR, d)
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
      mkdir(path.join(converted_d_path,'pmsys'))
    
    d_path = path.join(d_path, 'pmsys')
    converted_d_path = path.join(converted_d_path, 'pmsys')
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue
      
      #if path.exists(path.join(converted_d_path, f.split('.')[0] + '.csv')):
        #continue
      
      #print(str(f).split('.'))
      if str(f) == 'injury.csv':
        #print(f_path)
        convert_injury(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'),str(f))
      if str(f) == 'srpe.csv':
        convert_srpe(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'),str(f))
      if str(f) == 'wellness.csv':
        convert_srpe(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'),str(f))

#CONVERT FILE IN GOOGLEDOCS FOLDER

In [0]:
def convert_reporting(infile,outfile,filename):
  columns = ""
  filetxt = str(infile).replace(filename,'info_'+filename.split('.')[0]+'.txt')
  fi = open(filetxt,'r')
  for line in fi.readlines():
    columns += str(line)
  fi.close()
  columns = columns.replace('\n','').replace(' ','').replace('p,','p;').replace('d,','d;').replace('?,','?;').replace('s,','s;').replace('",','";').replace(',','|').split(';')

  df = pd.read_csv(infile)
  df.columns = columns
  for i in df.index:
    stemp = str(df.at[i,'Eatenmeals']).replace(',','|')
    df.at[i,'Eatenmeals'] = stemp
  df.to_csv(outfile,header=True,index=False)

In [0]:
def convert_googledocs_folder():
  #convert googledocs/[file]
  for d in listdir(DATA_DIR):
    d_path = path.join(DATA_DIR, d)
    if path.isfile(d_path):
      continue

    converted_d_path = path.join(CONVERTED_DIR, d)
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
      mkdir(path.join(converted_d_path,'googledocs'))
    
    d_path = path.join(d_path, 'googledocs')
    converted_d_path = path.join(converted_d_path, 'googledocs')
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue
      
      #if path.exists(path.join(converted_d_path, f.split('.')[0] + '.csv')):
        #continue
      
      #print(str(f).split('.'))
      if str(f) == 'reporting.csv':
        print(f_path)
        convert_reporting(f_path, path.join(converted_d_path, f.split('.')[0] + '.csv'),str(f))

#COPY ALL CSV FILE TO MERGE FOLDER

In [0]:
def copy_csv_merge_folder():
  for d in listdir(DATA_DIR):
    d_path = path.join(DATA_DIR, d)
    if path.isfile(d_path):
      continue

    converted_d_path = path.join(CONVERTED_DIR, d)
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
      mkdir(path.join(converted_d_path,'fitbit'))
    
    d_path = path.join(d_path, 'fitbit')
    converted_d_path = path.join(converted_d_path, 'fitbit')
    if not(path.exists(converted_d_path)):
      mkdir(converted_d_path)
    
    merged_d_path= path.join(MERGED_DIR, d)
    if not(path.exists(merged_d_path)):
      mkdir(merged_d_path)
    
    for f in listdir(d_path):
      f_path = path.join(d_path, f)
      if path.isdir(f):
        continue
      
      #if path.exists(path.join(converted_d_path, f.split('.')[0] + '.csv')):
        #continue
      
      #print(str(f).split('.'))
      if str(f).split('.')[-1] == 'csv':
        shutil.copyfile(f_path, path.join(merged_d_path, f.split('.')[0] + '.csv'))
        print(f_path)
      
    converted_d_path = path.join(CONVERTED_DIR, d)
    converted_d_path = path.join(converted_d_path, 'googledocs')
    merged_d_path= path.join(MERGED_DIR, d)
    if not(path.exists(merged_d_path)):
      mkdir(merged_d_path)
    for f in listdir(converted_d_path):
      f_path = path.join(converted_d_path, f)
      if path.isdir(f):
        continue
      
      #if path.exists(path.join(converted_d_path, f.split('.')[0] + '.csv')):
        #continue
      
      #print(str(f).split('.'))
      if str(f).split('.')[-1] == 'csv':
        shutil.copyfile(f_path, path.join(merged_d_path, f.split('.')[0] + '.csv'))
        print(f_path)
      
    converted_d_path = path.join(CONVERTED_DIR, d)
    converted_d_path = path.join(converted_d_path, 'pmsys')
    merged_d_path= path.join(MERGED_DIR, d)
    if not(path.exists(merged_d_path)):
      mkdir(merged_d_path)
    for f in listdir(converted_d_path):
      f_path = path.join(converted_d_path, f)
      if path.isdir(f):
        continue
      
      #if path.exists(path.join(converted_d_path, f.split('.')[0] + '.csv')):
        #continue
      
      #print(str(f).split('.'))
      if str(f).split('.')[-1] == 'csv':
        shutil.copyfile(f_path, path.join(merged_d_path, f.split('.')[0] + '.csv'))
        print(f_path)

#PREPROCESS CSV FILES

In [0]:
def pp_sleep(d_path,preprocessed_d_path):
  df = pd.read_csv(path.join(d_path,'sleep.csv'))
  
  for co_name in df.columns:
    if co_name.count('minutes') > 0:
      df[co_name] = df[co_name]*60
      df.rename(columns={co_name:co_name.replace('minutes','seconds')},inplace=True)
  df.rename(columns={'dateOfSleep':'date'},inplace=True)
  if df.empty:
    return df
  
  df.drop('startTime',axis='columns',inplace=True)
  df.endTime = df.endTime.str.replace('T', ' ').str.replace('.000','')
  df.endTime = pd.to_datetime(df.endTime).values.astype(np.int64) // 10 ** 9
  df.sort_values(by = 'endTime', ascending=False, inplace=True)
  
  #boolean = df['endTime'].duplicated().any()
  boolean = False
  if boolean:
    print(d_path)
    print("sleep duplicated")
    print(df[df.duplicated()])
    #print(df[df.duplicated(['endTime'])])
    print('---------------------------------2----------------------------------')
  return df

In [0]:
def pp_sleepscore(d_path,preprocessed_d_path):
  df_ss = pd.read_csv(path.join(d_path,'sleep_score.csv'))
  df_ss.rename(columns={"timestamp": 'endTime'},inplace=True)
  if df_ss.empty:
    return df_ss
  
  df_ss.drop(df_ss[df_ss['overall_score'].isna()].index,inplace=True)
  df_ss.drop('sleep_log_entry_id',axis = 'columns', inplace=True)
  df_ss.endTime = df_ss.endTime.str.replace('T', ' ').str.replace('.000','')
  df_ss.endTime = pd.to_datetime(df_ss.endTime).values.astype(np.int64) // 10 ** 9
  for co_name in df_ss.columns:
    if str(co_name).count('minutes')>0:
      df_ss[co_name] = df_ss[co_name]*60
  
  #boolean = df_ss['endTime'].duplicated().any()
  boolean = False
  if boolean:
    print(d_path)
    print("sleep score duplicated")
    print(df_ss[df_ss.duplicated()])
    #print(df_ss[df_ss.duplicated('endTime')])
    print('---------------------------------1bbbbbbbbbbbb----------------------------------')
  #print(df_ss)
  return df_ss

In [0]:
def pp_levelsleep(d_path,preprocessed_d_path):
  filepath = path.join(d_path,'level_sleep.csv')
  if (not path.exists(filepath)):
    return
  df_ls = pd.read_csv(path.join(d_path,'level_sleep.csv'))
  
  for co_name in df_ls.columns:
    if str(co_name).count('minutes')>0:
      df_ls[co_name] = df_ls[co_name]*60
      df_ls.rename(columns={co_name:co_name.replace('minutes','seconds')},inplace=True)
  df_ls.rename(columns={"dateOfSleep": 'date'},inplace=True)
  if df_ls.empty:
    df_ls.to_csv(path.join(preprocessed_d_path,'level_sleep.csv'))
    return df_ls
  
  #index_name = df_ls[df_ls['deep_count'].isna()].index
  #df_ls.drop(index_name,inplace=True)
  df_ls.sort_values(by = ['date','deep_thirty_day_avg_seconds'], ascending=False, inplace=True)
  df_ls.drop_duplicates(inplace=True)
  
  boolean = df_ls['date'].duplicated().any()
  boolean = False
  if boolean:
    print(d_path)
    print("level sleep duplicated")
    print(df_ls[df_ls.duplicated()])
    print(df_ls[df_ls.duplicated('date')])
    print('---------------------------------1aaaaaaaaaaaaa----------------------------------')
  df_ls = df_ls.loc[(df_ls['deep_thirty_day_avg_seconds'] > 0) | ~df_ls['date'].duplicated()]
  #print(df_ls[df_ls.duplicated('date')])
  df_ls['duration'] = 0
  for co_name in df_ls.columns:
    if 'seconds' in str(co_name) and not 'avg' in str(co_name):
      df_ls.duration +=df_ls[co_name]
  df_ls.to_csv(path.join(preprocessed_d_path,'level_sleep.csv'))
  return df_ls

In [0]:
def pp_sleep_sleepscore(d_path,preprocessed_d_path):
  if (path.exists(path.join(d_path,'sleep.csv')) ==False or path.exists(path.join(d_path,'sleep_score.csv')) == False or path.exists(path.join(d_path,'level_sleep.csv'))==False):
    return
  df = pp_sleep(d_path,preprocessed_d_path)

  df_ss = pp_sleepscore(d_path,preprocessed_d_path)
  
  
  #str_column_names = 'date,deep_count,deep_minutes,deep_thirty_day_avg_minutes,wake_count,wake_minutes,wake_thirty_day_avg_minutes,light_count,light_minutes,light_thirty_day_avg_minutes,rem_count,rem_minutes,rem_thirty_day_avg_minutes,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness'
  #column_names = str_column_names.split(',')
  #column_new_names = str_column_names.replace('minutes','seconds').split(',')

  df_result = df.merge(df_ss,how='outer',on='endTime')
  for co_name in df_result.columns:
    if str(co_name).count('minutes')>0:
      df_result.rename(columns={co_name:co_name.replace('minutes','seconds')},inplace=True)
  print(df_result)
  if df_result.empty:
    df_result.to_csv(path.join(preprocessed_d_path,'ssleep_score.csv'))
    return df_result
  df_result.sort_values(by = 'endTime', ascending=False, inplace=True)
  df_result.duration = df_result.duration/1000
  df_result.timeInBed = df_result.timeInBed*60
  df_result.drop_duplicates(inplace=True)
  
  #boolean = df_result['endTime'].duplicated().any()
  boolean = False
  if boolean:
    print("ssleep score duplicated")
    print(df_result[df_result.duplicated()])
    #print(df_result[df_result.duplicated('endTime')])
    print('---------------------------------1----------------------------------')
  df_result.to_csv(path.join(preprocessed_d_path,'ssleep_score.csv'))
  return df_result

join(level_sleep,ssleep_score) \
-deep_minutes = deep_sleep_in_seconds \
-wake_minutes = secondsAwake

In [0]:
def pp_data_sleep(d_path,preprocessed_d_path):
  if (path.exists(path.join(d_path,'sleep.csv')) ==False or path.exists(path.join(d_path,'sleep_score.csv')) == False or path.exists(path.join(d_path,'level_sleep.csv'))==False):
    return
  df_ls = pp_levelsleep(d_path,preprocessed_d_path)

  df_sss = pp_sleep_sleepscore(d_path,preprocessed_d_path)
  df_sss.rename(columns={'deep_sleep_in_seconds':'deep_seconds','secondsToFallAsleep':'fall_asleep_seconds','secondsAsleep':'asleep_seconds','endTime':'end_time','secondsAwake':'wake_seconds','secondsAfterWakeup':'after_wake_seconds','timeInBed':'in_bed_seconds','infoCode':'info_code','mainSleep':'main_sleep'},inplace=True)
  df_sss.drop(['deep_seconds','wake_seconds'],axis='columns',inplace=True)
  
  df_result = df_sss.merge(df_ls,how='outer',on=['date','duration'])
  if df_result.empty:
    df_result.to_csv(path.join(preprocessed_d_path,'sleep_data.csv'))
    return
  df_result.sort_values(by = 'date',  ascending=False, inplace=True)
  print('d_path: {}\ndf level sleep: {}\ndf ssleep core: {}\ndf result: {}\n'.format(d_path,len(df_ls.index),len(df_sss.index),len(df_result.index)))
  df_result.to_csv(path.join(preprocessed_d_path,'sleep_data.csv'))

In [0]:
def pp_data_exercise(d_path,preprocessed_d_path):
  file_path = path.join(d_path,'exercise.csv')
  print(file_path)
  if (path.exists(file_path)) ==False:
    return
  column_name = 'activityName,averageHeartRate,calories,distance,duration,steps,speed,startTime,elevationGain,sedentary_active_minutes,lightly_active_minutes,fairly_active_minutes,very_active_minutes,time_in_fat_burn_heart_rate_zone,time_in_out_of_range_heart_rate_zone,time_in_peak_heart_rate_zone,time_in_cardio_heart_rate_zone,vo2Max'.split(',')
  df = pd.read_csv(file_path)[column_name]
  for co_name in df.columns:
    if 'minutes' in co_name:
      df[co_name] = df[co_name]*60
      df.rename(columns={co_name:co_name.replace('minutes','seconds')},inplace=True)
  df.rename(columns={'activityName':'activity_name','averageHeartRate':'average_heart_rate','startTime':'start_time','elevationGain':'elevantion_gain'},inplace=True)
  df.duration = df.duration/1000
  #df.start_time = pd.to_datetime(df.start_time)
  #df.start_time = df.start_time.dt.strftime('%Y-%m-%d %H:%M:%S')
  df.start_time = pd.to_datetime(df.start_time).values.astype(np.int64) // 10 ** 9
  df.sort_values(by='start_time', ascending = False, inplace=True)
  new_column_name = 'injuries,fatigue,mood,readiness,calories_from_file,distance_from_file,steps_from_file,avg_heart_rate_from_file'.replace(' ','').replace('\n','').split(',')
  for co_name in new_column_name:
    df[co_name] = 0
  df['injuries'] = '{'
  
  file_path = path.join(d_path,'total_cal_dis_ste_hea.csv')
  df_tot = pd.read_csv(file_path)
  df_tot.dateTime = pd.to_datetime(df_tot.dateTime).values.astype(np.int64) // 10 ** 9
  df_tot.calories[pd.isna(df_tot.calories)] = 0
  df_tot.distance[pd.isna(df_tot.distance)] = 0
  df_tot.steps[pd.isna(df_tot.steps)] = 0
  df_tot.heart_rate[pd.isna(df_tot.heart_rate)] = 0
  df_tot.sort_values(by = 'dateTime',ascending=True, inplace=True)

  df_wel = []
  file_path = path.join(d_path,'wellness.csv')
  if (path.exists(file_path)):
    df_wel = pd.read_csv(file_path)
    df_wel.effective_time_frame = df_wel.effective_time_frame.str.replace('T',' ').str.split('.').str[0]
    df_wel.effective_time_frame = pd.to_datetime(df_wel.effective_time_frame).values.astype(np.int64) // 10 ** 9
    df_wel.sort_values(by = 'effective_time_frame',ascending=True, inplace=True)

  df_in = []
  file_path = path.join(d_path,'injury.csv')
  if (path.exists(file_path)):
    df_in = pd.read_csv(file_path)
    df_in.effective_time_frame = df_in.effective_time_frame.str.replace('T',' ').str.split('.').str[0]
    df_in.effective_time_frame = pd.to_datetime(df_in.effective_time_frame).values.astype(np.int64) // 10 ** 9
    df_in.sort_values(by = 'effective_time_frame',ascending=True, inplace=True)

  for i in range(len(df)):
    count = 0
    # for inde in range(len(df_tot)):
    #   #print(df_tot.iloc[inde])
    #   if df.at[i,'start_time'] + df.at[i,'duration'] < df_tot.at[inde,'dateTime']:
    #     break
    #   if df.at[i,'start_time'] <= df_tot.at[inde,'dateTime']:
    #     count +=1
    #     df.at[i,'calories_from_file'] += df_tot.at[inde,'calories']
    #     df.at[i,'distance_from_file'] += df_tot.at[inde,'distance']
    #     df.at[i,'steps_from_file'] += df_tot.at[inde,'steps']
    #     df.at[i,'avg_heart_rate_from_file'] += df_tot.at[inde,'heart_rate']
    # if count != 0:
    #   df.at[i,'avg_heart_rate_from_file'] = df.at[i,'avg_heart_rate_from_file'] / count
    
    # for inde in range(len(df_wel)):
    #   if df.at[i,'start_time'] + df.at[i,'duration'] < df_wel.at[inde,'effective_time_frame']:
    #     break
    #   if df.at[i,'start_time'] <= df_wel.at[inde,'effective_time_frame']:
    #     df.at[i,'fatigue'] += df_wel.at[inde,'fatigue']
    #     df.at[i,'mood'] += df_wel.at[inde,'mood']
    #     df.at[i,'readiness'] += df_wel.at[inde,'readiness']

    # for inde in range(len(df_in)):
    #   if df.at[i,'start_time'] + df.at[i,'duration'] < df_in.at[inde,'effective_time_frame']:
    #     break
    #   if df.at[i,'start_time'] <= df_in.at[inde,'effective_time_frame']:
    #     df.at[i,'injuries'] += df_in.at[inde,'injuries'].replace('{','').replace('}','') +'|'

    st = df.at[i,'start_time']
    en = df.at[i,'start_time'] + df.at[i,'duration']
    pst = 10**15
    #df_tot
    if (len(df_tot)>0):
      ar_tot = df_tot['dateTime'].to_numpy()
      pst = np.searchsorted(ar_tot, st)
    #print(ar_tot)
    #print(pst)
    for inde in range(pst,len(df_tot)):
      if en < df_tot.at[inde,'dateTime']:
        break
      count +=1
      df.at[i,'calories_from_file'] += df_tot.at[inde,'calories']
      df.at[i,'distance_from_file'] += df_tot.at[inde,'distance']
      df.at[i,'steps_from_file'] += df_tot.at[inde,'steps']
      df.at[i,'avg_heart_rate_from_file'] += df_tot.at[inde,'heart_rate']
    if count != 0:
      df.at[i,'avg_heart_rate_from_file'] = df.at[i,'avg_heart_rate_from_file'] / count
    
    pst = 10**15
    #df_wel
    if (len(df_wel)>0):
      ar_wel = df_wel['effective_time_frame'].to_numpy()
      pst = np.searchsorted(ar_wel,st)
    for inde in range(pst,len(df_wel)):
      if en < df_wel.at[inde,'effective_time_frame']:
        break
      df.at[i,'fatigue'] += df_wel.at[inde,'fatigue']
      df.at[i,'mood'] += df_wel.at[inde,'mood']
      df.at[i,'readiness'] += df_wel.at[inde,'readiness']
    
    pst = 10**15
    #df_in
    if (len(df_in)>0):
      ar_in = df_in['effective_time_frame'].to_numpy()
      pst = np.searchsorted(ar_in,st)
    for inde in range(pst,len(df_in)):
      if en < df_in.at[inde,'effective_time_frame']:
        break
      df.at[i,'injuries'] += df_in.at[inde,'injuries'].replace('{','').replace('}','') +'|'

  df.injuries +='}'
  df.to_csv(path.join(preprocessed_d_path,'exercise_data.csv'))

In [0]:
def pp_data_reporting(d_path,preprocessed_d_path):
  file_path = path.join(d_path,'reporting.csv')
  print(file_path)
  if (path.exists(file_path)) ==False:
    return
  df = pd.read_csv(file_path)
  df.drop(['Timestamp','unused'],axis='columns',inplace=True)
  df.rename(columns={'Whatdayyoureportfor?':'date','Eatenmeals':'meal','Whatisyourweight?':'weight','"Howmanyglassesoffluiddidyoudrink(water|milk|juice...)"':'glasses','Didyoudrinkalcohol?':'alcohol'},inplace=True)
  df.date = pd.to_datetime(df.date,format='%d/%m/%Y')
  df.date = pd.to_datetime(df.date,format='%Y-%m-%d')
  df.sort_values(by = 'date',ascending=False, inplace= True)
  new_column_names = 'very_active,moderately_active,lightly_active,sedentary,calories,distance,steps,heart_rate'.split(',')
  for co_name in new_column_names:
    df[co_name] = 0

  file_path = path.join(d_path,'total_cal_dis_ste_hea.csv')
  df_tot = pd.read_csv(file_path)
  df_tot.rename(columns={'dateTime':'date'},inplace=True)
  df_tot.date = df_tot.date.str.replace('\n','').str.split(' ').str[0]
  df_tot.date = pd.to_datetime(df_tot.date,format='%m/%d/%y')
  df_tot.date = pd.to_datetime(df_tot.date,format='%Y-%m-%d')
  #df_tot['count'] = 1
  df_tot.insert(0,'count',int(1))
  df_tot.count = df_tot['count'].astype('int64')
  df_tot = df_tot.groupby(['date']).sum()
  df_tot.count = df_tot['count'].astype('int64')
  #print(type(df_tot.count[0]))
  #print(pd.isna(df_tot.heart_rate).index)
  #df_tot.loc['heart_rate',pd.isna(df_tot.heart_rate).index] = 0
  df_tot.heart_rate[pd.isna(df_tot.heart_rate)] = 0
  #df_tot.heart_rate = df_tot.heart_rate/df_tot.count
  #print(type(df_tot.heart_rate[0]))
  df_tot.heart_rate = df_tot.heart_rate.div(df_tot.count,axis=0)
  df_tot.drop('count',axis='columns',inplace=True)
  df_tot.calories[pd.isna(df_tot.calories)] = 0
  df_tot.distance[pd.isna(df_tot.distance)] = 0
  df_tot.steps[pd.isna(df_tot.steps)] = 0
  df_tot.heart_rate[pd.isna(df_tot.heart_rate)] = 0
  # df_tot.loc['calories',pd.isna(df_tot.calories).index] = 0
  # df_tot.loc['distance',pd.isna(df_tot.distance).index] = 0
  # df_tot.loc['steps',pd.isna(df_tot.steps).index] = 0
  # df_tot.loc['heart_rate',pd.isna(df_tot.heart_rate).index] = 0
  

  file_path = path.join(d_path,'total_active_minutes.csv')
  df_tm = pd.read_csv(file_path)
  df_tm.rename(columns={'dateTime':'date'},inplace=True)
  df_tm.date = df_tm.date.str.split(' ').str[0]
  df_tm.date = pd.to_datetime(df_tm.date,format='%m/%d/%y')
  df_tm.date = pd.to_datetime(df_tm.date,format='%Y-%m-%d')
  for co_name in 'very_active,moderately_active,lightly_active,sedentary'.split(','):
    df_tm[co_name] = df_tm[co_name]*60
  df_tm = df_tm.groupby(['date']).sum()
  #print(df_tm.index)

  for co_name in new_column_names:
    #print(co_name)
    if co_name in df_tot.columns:
      for i in range(len(df)):
        if df.at[i,'date'] in df_tot.index:
          #print('df_tot')
          df.at[i,co_name] = df_tot.at[df.at[i,'date'],co_name]
      # if co_name == 'heart_rate':
      #   df[co_name] = df[co_name]/len(df_tot[co_name])
    if co_name in df_tm.columns:
      for i in range(len(df)):
        if df.at[i,'date'] in df_tm.index:
          #print('df_tm')
          df.at[i,co_name] = df_tm.at[df.at[i,'date'],co_name]
  
  df.to_csv(path.join(preprocessed_d_path,'report_data.csv'))

In [0]:
def preprocess_merged_data():
  for d in listdir(MERGED_DIR):
    d_path = path.join(MERGED_DIR,d)
    if path.isfile(d_path):
      continue
    
    preprocessed_d_path = path.join(PRE_PROCESSED_DIR,d)
    if not(path.exists(preprocessed_d_path)):
      mkdir(preprocessed_d_path)
     
    #pp_data_sleep(d_path,preprocessed_d_path)
    #pp_data_exercise(d_path,preprocessed_d_path)
    #pp_data_reporting(d_path,preprocessed_d_path)

In [0]:
def main():
  #convert_fitbit_folder()
  #merge_fitbit_folder()
  #merge_multi_file_type_of_fitbit_folder(MERGED_DIR, MERGED_DIR)
  #convert_pmsys_folder()
  #convert_googledocs_folder()
  #copy_csv_merge_folder()
  preprocess_merged_data()

In [55]:
main()
# if __name__ == 'main':
#   main()

./spll/data/merged/p09/exercise.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to b

./spll/data/merged/p09/reporting.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


./spll/data/merged/p04/exercise.csv
./spll/data/merged/p04/reporting.csv
./spll/data/merged/p03/exercise.csv
./spll/data/merged/p03/reporting.csv
./spll/data/merged/p12/exercise.csv
./spll/data/merged/p12/reporting.csv
./spll/data/merged/p11/exercise.csv
./spll/data/merged/p11/reporting.csv
./spll/data/merged/p07/exercise.csv
./spll/data/merged/p07/reporting.csv
./spll/data/merged/p02/exercise.csv
./spll/data/merged/p02/reporting.csv
./spll/data/merged/p06/exercise.csv
./spll/data/merged/p06/reporting.csv
./spll/data/merged/p10/exercise.csv
./spll/data/merged/p10/reporting.csv
./spll/data/merged/p05/exercise.csv
./spll/data/merged/p05/reporting.csv
./spll/data/merged/p01/exercise.csv
./spll/data/merged/p01/reporting.csv
./spll/data/merged/p08/exercise.csv
./spll/data/merged/p08/reporting.csv


In [0]:
#!zip -r /content/converted.zip /content/spll/data/converted
#from google.colab import files
#files.download("/content/converted.zip")

In [0]:
#!zip -r /content/merged.zip /content/spll/data/merged
#from google.colab import files
#files.download("/content/merged.zip")

In [58]:
!zip -r /content/preprocessed.zip /content/spll/data/preprocessed
from google.colab import files
files.download("/content/preprocessed.zip")

  adding: content/spll/data/preprocessed/ (stored 0%)
  adding: content/spll/data/preprocessed/p09/ (stored 0%)
  adding: content/spll/data/preprocessed/p09/level_sleep.csv (deflated 71%)
  adding: content/spll/data/preprocessed/p09/sleep_data.csv (deflated 70%)
  adding: content/spll/data/preprocessed/p09/report_data.csv (deflated 64%)
  adding: content/spll/data/preprocessed/p09/exercise_data.csv (deflated 64%)
  adding: content/spll/data/preprocessed/p09/ssleep_score.csv (deflated 67%)
  adding: content/spll/data/preprocessed/p04/ (stored 0%)
  adding: content/spll/data/preprocessed/p04/report_data.csv (deflated 69%)
  adding: content/spll/data/preprocessed/p04/exercise_data.csv (deflated 65%)
  adding: content/spll/data/preprocessed/p03/ (stored 0%)
  adding: content/spll/data/preprocessed/p03/report_data.csv (deflated 71%)
  adding: content/spll/data/preprocessed/p03/exercise_data.csv (deflated 66%)
  adding: content/spll/data/preprocessed/p12/ (stored 0%)
  adding: content/spll/d

MessageError: ignored